In [28]:
#Testing

# Explore here

In [29]:
# Your code here
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV

url = "https://raw.githubusercontent.com/4GeeksAcademy/logistic-regression-project-tutorial/main/bank-marketing-campaign-data.csv"
data = pd.read_csv(url, sep=";")
data.head(7)



,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
5,45,services,married,basic.9y,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
6,59,admin.,married,professional.course,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [30]:
print(data.shape)

(41188, 21)


In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [32]:
# Checking Balance
count = data['y'].value_counts(normalize=True)
print(count)

y
no     0.887346
yes    0.112654
Name: proportion, dtype: float64


In [33]:
# Transforming target variable y from categorical into binary
# Yes = 1, else (no) = 0

data['y'] = (data['y'] == 'yes').astype(int)

# Separating target(y) from the features (X)
X = data.drop(columns=['y'])
y = data['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(X_train.shape)
print(X_test.shape)

(32950, 20)
(8238, 20)


In [34]:
# Identifying numeral and categorical col names from the training data
num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
cat_cols = X_train.select_dtypes(include=['object']).columns

# Scale numerical data for model
# OneHotEncoder help me in two things, it will convert  categorical data into variables and with the "unkwown" feature, if the test has data the training didnt have. 
preprocessor = make_column_transformer(
    (StandardScaler(), num_cols),
    (OneHotEncoder(handle_unknown='ignore'), cat_cols),
    remainder='passthrough')

# Apply the preprocessor
# we do both fit and transform on the training data    
X_train_processed = preprocessor.fit_transform(X_train)
# We only transform the test data using the same transformer
X_test_processed = preprocessor.transform(X_test)

print(X_train_processed.shape)
print(X_test_processed)

(32950, 63)
[[-0.77033007 -0.49089273  0.88631588 ...  0.          1.
   0.        ]
 [-0.28972159 -0.6105968  -0.56702251 ...  0.          1.
   0.        ]
 [ 3.17065947 -0.49089273 -0.20368791 ...  0.          1.
   0.        ]
 ...
 [-0.67420837  0.825852   -0.56702251 ...  1.          0.
   0.        ]
 [ 0.38313029  1.58269061  1.61298507 ...  0.          1.
   0.        ]
 [ 0.19088689 -0.67624096  0.88631588 ...  0.          1.
   0.        ]]


In [35]:
# Train and check the model

# Basic LogisticRegression model
log_reg =LogisticRegression(random_state = 42, max_iter=1000)

# Train the model 
log_reg.fit(X_train_processed, y_train)

# Make predictions

y_pred = log_reg.predict(X_test_processed)

print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9166

Confusion Matrix:
 [[7146  164]
 [ 523  405]]

Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.98      0.95      7310
           1       0.71      0.44      0.54       928

    accuracy                           0.92      8238
   macro avg       0.82      0.71      0.75      8238
weighted avg       0.91      0.92      0.91      8238



In [38]:
print("Baseline Model Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Baseline Model Accuracy: 0.9166059723233795

Confusion Matrix:
[[7146  164]
 [ 523  405]]

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.98      0.95      7310
           1       0.71      0.44      0.54       928

    accuracy                           0.92      8238
   macro avg       0.82      0.71      0.75      8238
weighted avg       0.91      0.92      0.91      8238



Aqui el problema que podemos ver es que para los que no tienen sub, tenemos alto recall y precision pero no es lo mismo para los que si la tienen.
Por ende, nuestro modelo es solamente buen en detectar no subs.

Intentare optimizarlo usando classweight=balanced

In [37]:
# Try 2

# New model
log_reg_balanced = LogisticRegression(random_state = 42, class_weight = 'balanced', max_iter = 2000)

# Training
log_reg_balanced.fit(X_train_processed, y_train)

# Predict

y_pred_balanced = log_reg_balanced.predict(X_test_processed)

print("Balanced Model Accuracy:", accuracy_score(y_test, y_pred_balanced))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_balanced))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_balanced))



Balanced Model Accuracy: 0.8651371692158291

Confusion Matrix:
[[6281 1029]
 [  82  846]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.86      0.92      7310
           1       0.45      0.91      0.60       928

    accuracy                           0.87      8238
   macro avg       0.72      0.89      0.76      8238
weighted avg       0.93      0.87      0.88      8238

